In this blog post I'll explain how I created this blog, using Quarto, GitHub (and GitHub Pages). I'm working on a Macbook, and using VS Code for code editing. If you are on a Linux or Windows machine, be aware that things might be a bit different from what I describe here.

I am assuming you already have a GitHub account, that you have VS Code installed and set up in a way that you can run Python and Jupyter Notebooks.

## Step 1: install Quarto

First of all you need to install Quarto, head over to [here](https://quarto.org/docs/get-started/) and download and install the software. You should do this on the machine that you want to use for writing your blog, in my case my Macbook laptop.

Once installed you will have access to the quarto Command Line Interface (CLI). To make sure everything works as expected, open a terminal and execute:

```bash
quarto --help
```

This should give some outputs describing the different commands and options that are part of the Quarto CLI and shows that Quarto is installed successfully.


## Step 2: create a GitHub repo

To host our blog, we will use GitHub Pages, which is a service to host a website from a GitHub repository. Based on the name you pick for your repository you will create a so-called "project website" or your unique "user website". For any repo named `my-awesome-repo`, the website will be hosted on `https://<github-username>.github.io/my-awesome-repo`, this is what's called a project website and you can create as many as you like. 

You can create your "user website" by creating a repo called `<github-username>.github.io`, which will be hosted at `https://<github-username>.github.io`.

Since I want my blog to be hosted as "user website", I thus create a new repo with the name: `lucasvw.github.io`.

I find it helpful to add a `.gitignore` file with a Python template, to which we can later add some more entries to facilitate storing the right files on GitHub. Also make sure that the repo is Public (and not set to Private). Additionally, I added a README file and choose the Apache2 License.

Next, I clone this repo to my machine by running the following command in the folder where I have all my code project.

```bash
git clone git@github.com:lucasvw/lucasvw.github.io.git
```

## Step 3: create a Quarto project in the repo

Next, head over to VS Code, and open the cloned repo. Open the terminal inside of VS Code and run:

```bash
quarto create-project --type website:blog
```

This will add a number of files to our repo, which represent the basic structure of our blog. Most importantly:

- `posts`: here we will create our blog entries (one subfolder per blog entry)
- `_quarto.yml`: configuration file for our blog such as the theme, name, GitHub and Twitter links
- `about.qmd`: source code for the "about" page.
- `index.qmd`: source code for the landing page.

:::{.callout-note}
`.qmd` files are like markdown files, but with lots of additional functionality from Quarto. Go [here](https://www.markdownguide.org/basic-syntax/) for more information on Markdown syntax and [here](https://nbdev.fast.ai/tutorials/qmd_intro.html) for Quarto Markdown
:::

To see what we currently have, let's render our blog locally. We can do this by running

```bash
quarto preview
```

Alternatively, we can install the Quarto extension in VS Code, which will show a `render` button in the top right corner on any opened `qmd` file.

To publish to GitHub pages, we can run:

```bash
quarto publish gh-pages
```

When we run this, we get a message that we have to change the branch from which GitHub Pages builds the site. To do this, I go to [https://github.com/lucasvw/lucasvw.github.io/settings/pages](https://github.com/lucasvw/lucasvw.github.io/settings/pages) and select `gh-pages` instead of the `main` branch.

And voila, in a few moments our blog will be running live at [https://lucasvw.github.io/](https://lucasvw.github.io/)

## Step 4: finalize set-up and create first blog entry

When we run the `quarto publish gh-pages` command, Quarto processes our files and turns them into web readable files (HTML, JS, CSS etc). Next, it will push these files to the `gh-pages` branch. This is great, but it means that this doesn't store our source files in GitHub.

To do so, let's first open our `.gitignore` file and make sure that it contains the following entries:

```txt
# Quarto
/.quarto/
_site/

# Mac files
.DS_Store
```

Next, we can commit all the remaining files to Git and push them to our remote repo.

Finally, we can start tweaking our `about.qmd` files and create a new blog entry